[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/ds-reboot/python-first-part/blob/main/hometasks/Task1_dmrf.ipynb)

# Задание
* Собрать информацию о всех строящихся объектах на сайте "наш.дом.рф"
* Cохранить ее в pandas dataframe
* Cохранить pandas dataframe в excel
* Cохранить pandas dataframe в pickle
* Cохранить pandas dataframe в БД

# Задание *
* Написать скрипт, который скачивает фотографии всех строящихся объектов с сайта "наш.дом.рф"


In [1]:
import requests


## Список объектов

In [2]:
#Скачивание кусками в словарь
dct = {}
part_ = 1
for offset_ in range(1, 15000, 5000):
    limit_ = offset_ + 4999
    print(f'{offset_}:{limit_}')
    url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object?offset={offset_}&limit={limit_}&sortField=devId.devShortCleanNm&sortType=asc&objStatus=0'
    res = requests.get(url)
    data = res.json()
    dct.update({part_: data})
    part_ += 1


1:5000
5001:10000
10001:15000


In [30]:
#Все в один лист кидаем
lst_ = list()
for i in dct.keys():
    objects_list = dct[i].get('data').get('list')
    lst_ += objects_list
objids = [x.get('objId') for x in lst_]

# Данные по определенному объекту

In [101]:
# Для скачивания данных по определенному объекту, нужно добавить его id в конец строки запроса
import pandas as pd
pd_ = pd.DataFrame()
#j = 0
for i in objids:
    url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/{i}'
    res = requests.get(url)
    js_res = res.json()
    temp_pd = pd.json_normalize(js_res)
    pd_ = pd.concat([pd_,temp_pd])
    #j += 1
    #if j == 99: break
        

In [102]:
pd_

,errcode,data.id,data.pdId,data.developer.devId,data.developer.devShortCleanNm,data.developer.devShortNm,data.developer.devFullCleanNm,data.developer.problObjCnt,data.developer.buildObjCnt,data.developer.comissObjCnt,...,data.developer.bankruptStage.bankruptStageDesc,data.developer.bankruptStage.bankruptLawUrl,data.developer.orgBankruptMsgDttm,data.metro.id,data.metro.name,data.metro.line,data.metro.color,data.metro.time,data.metro.isWalk,data.metro.colors
0,0,29679,124647,7424,212-2,212-2,212-2,0,2,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,0,13383,28105,306,2МЕН ГРУПП,2МЕН ГРУПП,2МЕН ГРУПП,0,5,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,0,13391,28101,306,2МЕН ГРУПП,2МЕН ГРУПП,2МЕН ГРУПП,0,5,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,0,13397,28141,306,2МЕН ГРУПП,2МЕН ГРУПП,2МЕН ГРУПП,0,5,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,0,13398,28141,306,2МЕН ГРУПП,2МЕН ГРУПП,2МЕН ГРУПП,0,5,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,0,28070,99487,7344,ЯРКИЙ МИР,ЯРКИЙ МИР,ЯРКИЙ МИР,0,5,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,0,28072,96379,7344,ЯРКИЙ МИР,ЯРКИЙ МИР,ЯРКИЙ МИР,0,5,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,0,28073,96446,7344,ЯРКИЙ МИР,ЯРКИЙ МИР,ЯРКИЙ МИР,0,5,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,0,44447,475093,2538,ЯРОСЛАВЛЬЗАКАЗЧИК,ЯРОСЛАВЛЬЗАКАЗЧИК,СПЕЦИАЛИЗИРОВАННЫЙ ЗАСТРОЙЩИК ЯРОСЛАВЛЬЗАКАЗЧИК,0,1,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [118]:
pd_.to_excel('dom_rf.xlsx', index=False)

In [107]:
pd_.to_pickle("dom_rf.pkl")  

In [112]:
from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)

In [124]:
df = pd_

In [125]:
df = df.applymap(str)

In [128]:
df.to_sql('dom_rf', con=engine, if_exists='replace', index=False)

11052

In [131]:
engine.execute("SELECT * FROM dom_rf").fetchall()

[('0', '29679', '124647', '7424', '212-2', '212-2', '212-2', '0', '2', '0', 'Ростовская область', '+7(961)285-90-10', 'taganrogjsk.wixsite.com', 'kirpol76@gmail.com', '6154572922', '1116154005923', '615401001', 'Ростовская область, город Таганрог, улица Свободы/Трудовых резервов дом 25/9', 'обл Ростовская город Таганрог улица Сергея Шило д. 212-2', '126093', '0', '61.0', 'Снимщикова Ольга Викторовна', '20', 'Жилищный или жилищно-строительный кооператив', 'ЖСК', '0', '0', '61', 'г Таганрог', '18', '2020-03-31', 'Другое', 'Под ключ', 'Есть', '0', '1519.0', '61-000818', 'https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/file/092af1ad-2b49-4150-8a92-293031e2f79a', '20-03-2019 18:41', '47.2238', '38.8513', '0', '0', 'Комфорт', '15-08-2022 01:19', '[]', '0', 'normal', '1', '40800', '18', '1519.0', '0', '0', 'True', '0', '0', '0', '[]', 'False', '0', 'Не нормируется', '7', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan',